# Household Energy Consumption Patterns

This notebook focuses on energy consumption by households across EU countries, leveraging SDMX datasets.

We will show the energy consumption per household in each EU country. For that, we will use three datasets from Eurostat. On top of the energy balances and demographics datasets we have already seen, we will add a dataset with the average number of people living in a household. With that, we will be able to determine the number of households in each country, and obtain the per household consumption.

In [1]:
from utils import utils
import json
import plotly.express as px


with open('utils/sub_codelists.json', 'r') as f:
    sub_codelists = json.load(f)

Getting the datasets with the required constraints:

In [16]:
average_household_constraints = {
    'freq': 'A',
    'unit': 'AVG',
    'geo': sub_codelists['geo_eu_countries']
}

nrg_bal_constraints = {
    'freq': 'A',
    'nrg_bal': 'FC_OTH_HH_E',
    'siec': 'TOTAL',
    'unit': 'GWH',
    'geo': sub_codelists['geo_eu_countries'],
}

demo_pjan_constraints = {
    'freq': 'A',
    'unit': 'NR',
    'age': 'TOTAL',
    'sex': 'T',
    'geo': sub_codelists['geo_eu_countries'],
    }

average_household = utils.get_dataset_with_selection('ilc_lvph01', average_household_constraints, agency='estat')

demo_pjan = utils.get_dataset_with_selection('demo_pjan', demo_pjan_constraints, agency='estat')
demo_pjan.data.TIME_PERIOD  = demo_pjan.data.TIME_PERIOD  - 1

nrg_bal = utils.get_dataset_with_selection('nrg_bal_s', nrg_bal_constraints, agency='estat')

data_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/ilc_lvph01/A.AVG.BE+BG+CZ+DK+DE+EE+IE+EL+ES+FR+HR+IT+CY+LV+LT+LU+HU+MT+NL+AT+PL+PT+RO+SI+SK+FI+SE?format=SDMX-CSV&startPeriod=2000
metadata_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/dataflow/all/ilc_lvph01/latest?detail=full&references=descendants
data_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/demo_pjan/A.NR.TOTAL.T.BE+BG+CZ+DK+DE+EE+IE+EL+ES+FR+HR+IT+CY+LV+LT+LU+HU+MT+NL+AT+PL+PT+RO+SI+SK+FI+SE?format=SDMX-CSV&startPeriod=2000
metadata_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/dataflow/all/demo_pjan/latest?detail=full&references=descendants
data_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/nrg_bal_s/A.FC_OTH_HH_E.TOTAL.GWH.BE+BG+CZ+DK+DE+EE+IE+EL+ES+FR+HR+IT+CY+LV+LT+LU+HU+MT+NL+AT+PL+PT+RO+SI+SK+FI+SE?format=SDMX-CSV&startPeriod=2000
metadata_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/dataflow/all/nrg_bal_s/l

From these datasets, we calculate the consumption per household in MWh

In [13]:
#The demo_pjan dataset has the total population per country and year
demo_pjan_df = demo_pjan.data[['geo', 'TIME_PERIOD', 'OBS_VALUE']]
demo_pjan_df.set_index(['geo', 'TIME_PERIOD'], inplace=True)

#The average_household dataset has the average number of people per household per country and year
average_household_df = average_household.data[['geo', 'TIME_PERIOD', 'OBS_VALUE']]
average_household_df.set_index(['geo', 'TIME_PERIOD'], inplace=True)

#The number of households is the total population divided by the average number of people per household
number_of_households_df = demo_pjan_df.div(average_household_df)

#The nrg_bal dataset has the total energy consumption per country and year
nrg_bal_df = nrg_bal.data.set_index(['geo', 'TIME_PERIOD'])

#The consumption per household is the total energy consumption divided by the number of households
consumption_per_household = nrg_bal_df.div(number_of_households_df)
consumption_per_household.reset_index(inplace=True)
#Convert to MWh
consumption_per_household['OBS_VALUE'] = consumption_per_household['OBS_VALUE'] * 1000
consumption_per_household = consumption_per_household[consumption_per_household['TIME_PERIOD'] >= 2005]
consumption_per_household.rename(columns={
    'OBS_VALUE': 'Consumption per household (MWh)',
    'geo': 'Country',
    'TIME_PERIOD': 'Year'
    }, inplace=True)
fig = px.line(consumption_per_household, x='Year', y='Consumption per household (MWh)', color='Country')
fig.show()


This chart displays household energy consumption in megawatt-hours (MWh) across different European countries from 2005 to 2023.
Key observations:

- Luxembourg (LU, light green line at top)shows the highest household energy consumption in the beginning of the series, starting at around 32 MWh per household in 2005, but declines sharply converging to the mean.
- Finland (FI, blue line) demonstrates significant fluctuations but maintains relatively high consumption between 20-26 MWh per household
Most countries cluster in the 10-20 MWh range with varying trajectories.
- There's a general downward trend across most countries, particularly after 2010
- Several countries show temporary increases around 2009-2010 before declining
- The lowest-consuming countries (bottom of chart) maintain relatively stable patterns around 5-10 MWh per household

The visualization illustrates the significant differences in household energy consumption between European nations, likely reflecting variations in climate conditions, housing efficiency, heating/cooling needs, appliance usage, and energy conservation policies. The overall downward trend suggests improvements in energy efficiency across European households over this period.

## Testing the climate conditions hypothesis

In order to test that hypothesis, we are going to get a dataset that includes the cooling and heating degree days indexes (see: https://ec.europa.eu/eurostat/cache/metadata/en/nrg_chdd_esms.htm)

In [14]:
nrg_chdd_a_constraints = {
    'freq': 'A',
    'unit': 'NR',
    'indic_nrg': ['CDD', 'HDD'],
    'geo': sub_codelists['geo_eu_countries'],
    }

nrg_chdd_a_constraints = utils.get_dataset_with_selection('nrg_chdd_a', nrg_chdd_a_constraints, agency='estat')
nrg_chdd_a_constraints_df = nrg_chdd_a_constraints.data[['geo', 'TIME_PERIOD', 'indic_nrg', 'OBS_VALUE']]


data_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/nrg_chdd_a/A.NR.CDD+HDD.BE+BG+CZ+DK+DE+EE+IE+EL+ES+FR+HR+IT+CY+LV+LT+LU+HU+MT+NL+AT+PL+PT+RO+SI+SK+FI+SE?format=SDMX-CSV&startPeriod=2000
metadata_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/dataflow/all/nrg_chdd_a/latest?detail=full&references=descendants


In [36]:
# nrg_chdd_a_constraints_df.set_index(['geo', 'TIME_PERIOD', 'indic_nrg'], inplace=True)
nrg_chdd_a_constraints_df = nrg_chdd_a_constraints.data[['geo', 'TIME_PERIOD', 'indic_nrg', 'OBS_VALUE']]
nrg_chdd_a_constraints_df = nrg_chdd_a_constraints_df.pivot(columns=['indic_nrg'], index=['geo', 'TIME_PERIOD'], values='OBS_VALUE')

#We are calculating a single index for each country and year, which is the sum of the cooling and heating degree days indexes
#multiplied by a factor, because heating is more energy intensive than cooling. We are using a factor
#Taken form https://www.construction-physics.com/p/energy-cheat-sheet, US Residential Energy Consumption table
#There we can see that heating is (5.74 - 0.91) / 0.91 = 5.3 times more energy intensive than cooling
nrg_chdd_a_constraints_df['Temperature energy index'] = nrg_chdd_a_constraints_df['CDD']  + nrg_chdd_a_constraints_df['HDD'] * 5.3
nrg_chdd_a_constraints_df = nrg_chdd_a_constraints_df.reset_index()

#And now we merge the consumption per household with the total degree days
consumption_per_household_chdd = consumption_per_household.merge(nrg_chdd_a_constraints_df, left_index=True, right_index=True)

# #We can now plot the consumption per household against the total degree days
fig = px.scatter(consumption_per_household_chdd, x='Temperature energy index', y='Consumption per household (MWh)', color='Country')
fig.show()


With this plot we can see the expected relationship in the extremes. Malta (MT) and Finland (FI) shows a good correlation betwee the variables. 

There are, however several outlier that deserve attention. For instance, Spain (ES) shows a relatively low consumption for the temperature energy index. The high temeperature energy index for Spain (the highest in the EU? it is counterintuitive) may suggest that the extreme temperatures in the country are taken. Spain is a big country, with very cold and very warm areas. So, if the coldest areas are used for calculating the index, this may provide a distorted immage at country level.

It is worth noting that, given each individual country, the correlation betwween the temperature energy index and the comsumption per household is clearly positive, except in relatively warm countries (like Malta and Portugal), which suggests that households may not have heating systems because they are normally not necessary.